In [13]:
import pandas as pd
import logging 
import numpy as np
import re 
from IPython.display import display
from sqlalchemy import MetaData, Table
from database_utils import DatabaseConnector
from data_extraction import DataExtractor
from unidecode import unidecode
from dateutil import parser

In [ ]:
# this is the last one to try and combine. The code has been tested and works. 
# But I haven't brought it together with the code in the daa_cleaning file. After that, it should all wrok
# missing expiry date here at the start 

In [14]:
instance = DataExtractor()

        # Read data from the 'legacy_users' table
df = instance.retrieve_pdf_data('https://data-handling-public.s3.eu-west-1.amazonaws.com/card_details.pdf')

        

retrieve_pdf_data is working


In [15]:
df.info() 

display(len((df['card_number'].unique()))) 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15309 entries, 0 to 15308
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   card_number             15309 non-null  object
 1   expiry_date             15309 non-null  object
 2   card_provider           15309 non-null  object
 3   date_payment_confirmed  15309 non-null  object
dtypes: object(4)
memory usage: 478.5+ KB


15299

In [16]:
df_backup = df
df.info()
df_backup.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15309 entries, 0 to 15308
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   card_number             15309 non-null  object
 1   expiry_date             15309 non-null  object
 2   card_provider           15309 non-null  object
 3   date_payment_confirmed  15309 non-null  object
dtypes: object(4)
memory usage: 478.5+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15309 entries, 0 to 15308
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   card_number             15309 non-null  object
 1   expiry_date             15309 non-null  object
 2   card_provider           15309 non-null  object
 3   date_payment_confirmed  15309 non-null  object
dtypes: object(4)
memory usage: 478.5+ KB


In [30]:
df = df_backup
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15309 entries, 0 to 15308
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   card_number             15309 non-null  object
 1   expiry_date             15309 non-null  object
 2   card_provider           15309 non-null  object
 3   date_payment_confirmed  15309 non-null  object
dtypes: object(4)
memory usage: 478.5+ KB


In [31]:

#log starting of method
#self.logger.info(f"Running retrieve_pdf_data with pdf_path: {pdf_path}")

#cleaning expiry date column 


# STEP 1: Cleaning expiry date column
print('started cleaning expiry date column')

# Define a regular expression pattern for the MM/YY format
pattern_exp = r'^\d{2}/\d{2}$'

# Use str.match to filter rows where 'expiry_date' matches the pattern
df = df[df['expiry_date'].str.match(pattern_exp)].reset_index(drop=True)

print('started datetime conversion for expiry_date')

# Convert expiry_date to datetime, coerce errors to NaT using .loc to avoid SettingWithCopyWarning
df.loc[:, 'datetime_expiry_date'] = pd.to_datetime(df['expiry_date'], format='%m/%y', errors='coerce')

# Identify rows that will be removed (rows with NaT in 'datetime_expiry_date')
removed_expiry_dates = df[df['datetime_expiry_date'].isna()]

print("Expiry date datetime conversion that will be removed:")
display(removed_expiry_dates)

# Reset the index
df = df.reset_index(drop=True)

num_rows = df.shape[0]
print(f"Number of rows after cleaning expiry date: {num_rows}")

# STEP 2: Cleaning card number
num_rows = df.shape[0]
print(f"Number of rows before card number cleaning: {num_rows}")

# Define a regular expression pattern for numbers only
pattern_card = r'^\d+$'

print("Number of non matches:", len(non_matching_cards))
print("Non-matching cards:", non_matching_cards)

# Ensure all elements in the 'card_number' column are strings
df['card_number'] = df['card_number'].astype(str)

# Remove '??' from the strings in the 'card_number' column
df['card_number'] = df['card_number'].str.replace('?', '', regex=False)

# Use apply with a lambda function to filter rows where 'card_number' matches the pattern
df = df[df['card_number'].apply(lambda x: bool(re.match(pattern_card, x)))].reset_index(drop=True)

# STEP 3: Cleaning date_payment_confirmed
num_rows = df.shape[0]
print(f"Number of rows before date_payment_confirmed cleaning: {num_rows}")

# Initialize a list to store invalid dates
invalid_dates_list = []

# Function to parse dates and standardize format
def parse_date(date_str):
    try:
        # Attempt to parse the date string to a datetime object
        dt = parser.parse(date_str)
        # Convert to the desired format (YYYY-MM-DD)
        return dt.strftime('%Y-%m-%d')
    except (parser.ParserError, ValueError):
        # Append invalid date to the list
        invalid_dates_list.append(date_str)
        return np.nan  # Return NaN for invalid dates

# Apply the function to the 'date_payment_confirmed' column
df.loc[:, 'date_payment_confirmed'] = df['date_payment_confirmed'].apply(parse_date)

# Identify rows that would be null after conversion
invalid_rows = df[df['date_payment_confirmed'].isna()]

display("Rows that would be converted to NULL:")
display(invalid_rows)

# Drop rows with NaN (invalid dates)
df_cleaned = df.dropna(subset=['date_payment_confirmed'])

display("\nList of invalid dates:")
display(invalid_dates_list)

num_rows = df.shape[0]
print(f"Number of rows after date_payment_confirmed cleaning: {num_rows}")


# STEP 5: removing incorrect values from the card providers column 

# LOGGING 
print('started removing incorrect values from card providers column')

num_rows = df.shape[0]
print(f"Number of rows after expiry date to date time cleaning: {num_rows}")

#create a valid providers list  
valid_providers = ['Diners Club / Carte Blanche', 'American Express', 'JCB 16 digit','JCB 15 digit', 'Maestro', 'Mastercard', 'Discover','VISA 19 digit', 'VISA 16 digit', 'VISA 13 digit']

# Filter the DataFrame to keep only rows with valid card providers
df = df[df['card_provider'].isin(valid_providers)]

removed_providers = df[~df['card_provider'].isin(valid_providers)]
display("Card provider rows that will be removed:")
display(removed_expiry_dates)

#Reset the index of the filtered DataFrame
df = df.reset_index(drop=True)

num_rows = df.shape[0]
print(f"Number of rows after card number cleaning: {num_rows}")

started cleaning expiry date column
started datetime conversion for expiry_date
Expiry date datetime conversion that will be removed:


,card_number,expiry_date,card_provider,date_payment_confirmed,datetime_expiry_date


Number of rows after cleaning expiry date: 15284
Number of rows before card number cleaning: 15284
Number of non matches: 0
Non-matching cards: []
Number of rows before date_payment_confirmed cleaning: 15284


'Rows that would be converted to NULL:'

,card_number,expiry_date,card_provider,date_payment_confirmed,datetime_expiry_date


...

[]

Number of rows after date_payment_confirmed cleaning: 15284
started removing incorrect values from card providers column
Number of rows after expiry date to date time cleaning: 15284


'Card provider rows that will be removed:'

,card_number,expiry_date,card_provider,date_payment_confirmed,datetime_expiry_date


Number of rows after card number cleaning: 15284
